# Training Dialogue Encoder

## Data Validation

## MultiWOZ Service Classification

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
hidden_size = 768
n_classes = 7

train_size = 8000
val_size = 1000

In [3]:
import torch
from torch import nn
import torch.nn.functional as F


@torch.no_grad()
def generate_dataset(train_size, val_size, hidden_size, n_classes):
    embeddings = torch.empty(train_size+val_size, hidden_size)
    embeddings.normal_()

    linear_mapper = nn.Linear(hidden_size, n_classes)
    logits = linear_mapper(embeddings)
    targets = logits.ge(0).to(torch.float)

    train_dataset = list(zip(embeddings[:train_size], targets[:train_size]))
    val_dataset = list(zip(embeddings[train_size:], targets[train_size:]))

    return train_dataset, val_dataset

train_dataset, val_dataset = generate_dataset(train_size, val_size, hidden_size, n_classes)

In [ ]:
from train_utils import get_multiwoz_service_clf_score

score = get_multiwoz_service_clf_score(train_dataset, val_dataset, 'cuda')

In [5]:
score

tensor(0.8684, device='cuda:0')

### MLPClassifier

In [4]:
train_dataset, val_dataset = generate_dataset(train_size, val_size, hidden_size, n_classes)

In [5]:
import numpy as np
X_train = np.stack([emb for emb, _ in train_dataset], axis=0)
y_train = np.stack([tar for _, tar in train_dataset], axis=0)
X_val = np.stack([emb for emb, _ in val_dataset], axis=0)
y_val = np.stack([tar for _, tar in val_dataset], axis=0)

In [10]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(
    batch_size=32,
    learning_rate_init=5e-4,
    max_iter=5
)

clf.fit(X_train, y_train)

/home/alekseev_ilya/dialogue-augmentation/VENV/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(batch_size=32, learning_rate_init=0.0005, max_iter=5)

In [11]:
y_pred = clf.predict(X_val)
y_pred.shape

(1000, 7)

In [12]:
from sklearn.metrics import f1_score


f1_score(y_val, y_pred, average='macro')

0.8910066481802212